# Choropleth Mapy by using GPKG File 

This notebook is aming to explore the gpkg file and use its information to create choropleth map.

Obejectives:
- Exploring the GPKG file from GreenSpace datasets
- Create choropleth map by using converted gpkg file and merged datasets


Conclusion:
- The file can be read by geopandas. 
- The ...V1_2.gpkg contains MULTIPOLYGON data, can be used as area boundary for the choropleth map.

*Note: ...V1_2.gpkg file is very large, so it may take few minutes to load.*

In [1]:
import GeoBound_ChoroplethMap as gcm
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import numpy as np
import os

## Explore GPKG

In [2]:
gpkg_path = 'GreenspaceDownload/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg'

def load_gpkg(gpkg_path):
    print('Loading large file, will take 1-2 minutes...')
    gdf = gpd.read_file(gpkg_path)
    return gdf

In [3]:
gdf = load_gpkg(gpkg_path)
gdf.head()

Loading large file, will take 1-2 minutes...


,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,CTR_MN_NM,...,EX_SS_P00,EX_SS_P15,EX_EQ19PGA,EX_EQ19MMI,EX_EQ19_Q,EX_HW_IDX,SDG_LUE9015,SDG_A2G14,SDG_OS15MX,geometry
0,1.0,1.0,185.0,21.247683,-158.043016,21.422193,-157.730529,21.340678,-157.893497,United States,...,397443.031445,444041.529529,NaN,NaN,missing,NaN,0.074385,0.226415,56.41,"MULTIPOLYGON (((-158.01244 21.42219, -157.9915..."
1,2.0,2.0,42.0,-17.641184,-149.628088,-17.517631,-149.508018,-17.534103,-149.568053,French Polynesia,...,0.000000,0.000000,NaN,NaN,missing,NaN,0.128,0.284119,NaN,"MULTIPOLYGON (((-149.56967 -17.51763, -149.508..."
2,3.0,1.0,55.0,34.858517,-120.475511,34.989334,-120.389183,34.923123,-120.434372,United States,...,0.000000,0.000000,0.0,0.0,available,2.79174,0.48114,0.040129,23.64,"MULTIPOLYGON (((-120.46375 34.98933, -120.4411..."
3,4.0,1.0,48.0,36.582997,-121.952215,36.635743,-121.811816,36.607720,-121.882378,United States,...,0.000000,0.000000,0.0,0.0,available,NaN,0.44484,0.138683,42.17,"MULTIPOLYGON (((-121.95221 36.63574, -121.9179..."
4,5.0,1.0,60.0,34.388220,-119.853855,34.457831,-119.658413,34.427664,-119.743693,United States,...,0.000000,0.000000,0.0,0.0,available,4.25502,0.55676,0.061348,36.50,"MULTIPOLYGON (((-119.82444 34.45783, -119.8131..."


The file V1_2.gpkg contains columns that are almost identical to those in the greenspace CSV file, with the addition of a geometry column containing MULTIPOLYGON data.

However, this file takes a long time to be loaded due to its size, it is better to output it as a smaller, more readable geojson file. Additionally, we should only retain the data we are interested in -- data about the United States.

Hence, in the following code, I will filter out data that is not related to the United States and output the result as a geojson file named `Greenspace_US.geojson`.

In [4]:
gdf.head()

,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,CTR_MN_NM,...,EX_SS_P00,EX_SS_P15,EX_EQ19PGA,EX_EQ19MMI,EX_EQ19_Q,EX_HW_IDX,SDG_LUE9015,SDG_A2G14,SDG_OS15MX,geometry
0,1.0,1.0,185.0,21.247683,-158.043016,21.422193,-157.730529,21.340678,-157.893497,United States,...,397443.031445,444041.529529,NaN,NaN,missing,NaN,0.074385,0.226415,56.41,"MULTIPOLYGON (((-158.01244 21.42219, -157.9915..."
1,2.0,2.0,42.0,-17.641184,-149.628088,-17.517631,-149.508018,-17.534103,-149.568053,French Polynesia,...,0.000000,0.000000,NaN,NaN,missing,NaN,0.128,0.284119,NaN,"MULTIPOLYGON (((-149.56967 -17.51763, -149.508..."
2,3.0,1.0,55.0,34.858517,-120.475511,34.989334,-120.389183,34.923123,-120.434372,United States,...,0.000000,0.000000,0.0,0.0,available,2.79174,0.48114,0.040129,23.64,"MULTIPOLYGON (((-120.46375 34.98933, -120.4411..."
3,4.0,1.0,48.0,36.582997,-121.952215,36.635743,-121.811816,36.607720,-121.882378,United States,...,0.000000,0.000000,0.0,0.0,available,NaN,0.44484,0.138683,42.17,"MULTIPOLYGON (((-121.95221 36.63574, -121.9179..."
4,5.0,1.0,60.0,34.388220,-119.853855,34.457831,-119.658413,34.427664,-119.743693,United States,...,0.000000,0.000000,0.0,0.0,available,4.25502,0.55676,0.061348,36.50,"MULTIPOLYGON (((-119.82444 34.45783, -119.8131..."


In [5]:
# original has O'Fallon
gdf[gdf['UC_NM_MN'].str.contains("Fallon")]

,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,CTR_MN_NM,...,EX_SS_P00,EX_SS_P15,EX_EQ19PGA,EX_EQ19MMI,EX_EQ19_Q,EX_HW_IDX,SDG_LUE9015,SDG_A2G14,SDG_OS15MX,geometry
482,483.0,1.0,168.0,38.721506,-90.767949,38.837051,-90.469673,38.777312,-90.611861,United States,...,0.0,0.0,0.048507,4.0,available,17.7047,1.5328,0.790093,75.6,"MULTIPOLYGON (((-90.71472 38.83705, -90.69142 ..."


In [6]:
# helping functions for Greenspace_Data_Cleaning below

stateboundaries = gpd.read_file('cb_2018_us_state_500k/cb_2018_us_state_500k.shp')

def statefinder(row):
    point = Point(row['Longitude'], row['Latitude'])
    state = stateboundaries[stateboundaries.contains(point)]
    
    if not state.empty:
        return state.iloc[0]['STUSPS'] 
    else:
        return np.nan

In [7]:
# import code from Greenspace_Data_Cleaning.ipynb to generate UC_Grouping coloumn
# which is the index after cleaned the whole dataset and exploded the Cities in Urban Center column

def Greenspace_Data_Cleaning(rawdf):

    cols_to_keep = ['GCPNT_LAT', 'GCPNT_LON', 'CTR_MN_NM', 'UC_NM_MN', 'UC_NM_LST', 'E_GR_AV14', 'E_GR_AT14', 'SDG_A2G14', 'SDG_OS15MX', 'P15', 'B15', 'BUCAP15', 'INCM_CMI', 'DEV_CMI', 'GDP15_SM', 'E_BM_NM_LST', 'E_WR_T_14','geometry'] # add 'geometry' to keep the geometry column

    df = rawdf[cols_to_keep]
    df = df[df['CTR_MN_NM'] == 'United States']
    df.replace(to_replace=['?', '??', '???', 'NAN'], value = [np.nan, np.nan, np.nan, np.nan], inplace=True)
    df.rename(columns={'GCPNT_LAT': 'Latitude', 'GCPNT_LON': 'Longitude', 'CTR_MN_NM': 'Country', 'UC_NM_MN': 'Urban Center', 'UC_NM_LST': 'Cities in Urban Center'}, inplace=True)

    a1 = df.loc[482]['Cities in Urban Center']
    a1replace = a1.replace('’', "'")

    df.at[482, 'Urban Center'] = "O'Fallon"
    df.at[482, 'Cities in Urban Center'] = a1replace

    df['Cities in Urban Center_copy'] = df['Cities in Urban Center']
    df['Cities in Urban Center'] = df['Cities in Urban Center'].str.split(';')
    df = df.explode('Cities in Urban Center')
    df.reset_index(inplace=True, drop=False)
    df.rename(columns={'index': 'UC_Grouping'}, inplace=True) # update UC Grouping to UC_Grouping
    df['Cities in Urban Center'] = df['Cities in Urban Center'].str.strip()

    mhdf = pd.read_csv('MHDS/Original/500_Cities__City-level_Data__GIS_Friendly_Format___2017_release_20240514.csv')
    mh_cities = (mhdf['PlaceName'].unique()).tolist()

    ucgroup = df[df['Cities in Urban Center'].isin(mh_cities)]
    ucgrouplist = ucgroup.index.tolist()

    df = df[df.index.isin(ucgrouplist)]

    df['State'] = df.apply(statefinder, axis=1)

    return df


In [8]:
green_us = Greenspace_Data_Cleaning(gdf)
print(green_us.shape)
green_us.head(3)


(354, 21)


,UC_Grouping,Latitude,Longitude,Country,Urban Center,Cities in Urban Center,E_GR_AV14,E_GR_AT14,SDG_A2G14,SDG_OS15MX,...,B15,BUCAP15,INCM_CMI,DEV_CMI,GDP15_SM,E_BM_NM_LST,E_WR_T_14,geometry,Cities in Urban Center_copy,State
0,0,21.340678,-157.893497,United States,Honolulu,Honolulu,0.369290,183.811667,0.226415,56.41,...,80.647377,157.252219,HIC,MDR,2.192668e+10,Tropical and Subtropical Dry Broadleaf Forests,23.526622,"MULTIPOLYGON (((-158.01244 21.42219, -157.9915...",Honolulu; Waipahu; Pearl City; Aiea,HI
4,2,34.923123,-120.434372,United States,Santa Maria,Santa Maria,0.312846,54.450694,0.040129,23.64,...,42.000805,340.967420,HIC,MDR,4.174295e+09,"Mediterranean Forests, Woodlands, and Scrub",14.718191,"MULTIPOLYGON (((-120.46375 34.98933, -120.4411...",Santa Maria,CA
6,4,34.427664,-119.743693,United States,Santa Barbara,Santa Barbara,0.362785,59.576284,0.061348,36.50,...,38.101749,332.032274,HIC,MDR,4.159702e+09,"Mediterranean Forests, Woodlands, and Scrub",15.376907,"MULTIPOLYGON (((-119.82444 34.45783, -119.8131...",Santa Barbara,CA


After cleaned the geometry file and generate a dataframe `green_us` based on Greenspace_Data_Cleaning.ipynb, we need to conduct a further cleaning to make sure the geometry data align with the merged dataset.

So we will load the merged dataset and manipulated `green_us` for further cleaning.

In [9]:
file_path = 'uc_group_merged_greenspace_mh.csv'

In [10]:
# load merged dataset and manipulate rows to match the merged dataset
def rows_matching_with_merged(path, df = green_us, key_col = 'UC_Grouping'):
    mer_df = pd.read_csv(path,index_col = 0)

    key_list = (mer_df[key_col].unique()).tolist()

    matchgroup = df[df[key_col].isin(key_list)]
    matchgrouplist = matchgroup.index.tolist()

    df = df[df.index.isin(matchgrouplist)]

    return mer_df, df

In [11]:
uc_merged, up_greenus = rows_matching_with_merged(file_path, df = green_us, key_col = 'UC_Grouping')

# presenting merged dataframe
print(uc_merged.shape)
uc_merged.head(3)

(329, 19)


,Population2010,MHLTH_AdjPrev,UC_Grouping,Latitude,Longitude,E_GR_AV14,E_GR_AT14,SDG_A2G14,SDG_OS15MX,P15,B15,BUCAP15,GDP15_SM,E_WR_T_14,State,INCM_CMI,DEV_CMI,E_BM_NM_LST,Cities in Urban Center_copy
0,212237,15.6,485,33.509025,-86.823651,0.494568,219.996230,0.773812,74.85,196387.76700,152.894608,778.534274,6.184143e+09,17.497644,AL,HIC,MDR,Temperate Broadleaf and Mixed Forests,Birmingham;
1,180105,13.4,501,34.726065,-86.609995,0.521522,88.700999,0.802599,66.37,86467.06209,59.674004,690.135667,2.498489e+09,16.321889,AL,HIC,MDR,Temperate Broadleaf and Mixed Forests,Huntsville
2,195111,15.0,422,30.692377,-88.093685,0.467515,122.669298,0.822213,63.32,118578.67890,71.298004,601.271703,4.072112e+09,20.312027,AL,HIC,MDR,Temperate Coniferous Forests,Mobile


In [12]:
# presenting updated green_us dataframe
up_greenus.shape
up_greenus.head()

,UC_Grouping,Latitude,Longitude,Country,Urban Center,Cities in Urban Center,E_GR_AV14,E_GR_AT14,SDG_A2G14,SDG_OS15MX,...,B15,BUCAP15,INCM_CMI,DEV_CMI,GDP15_SM,E_BM_NM_LST,E_WR_T_14,geometry,Cities in Urban Center_copy,State
0,0,21.340678,-157.893497,United States,Honolulu,Honolulu,0.369290,183.811667,0.226415,56.41,...,80.647377,157.252219,HIC,MDR,2.192668e+10,Tropical and Subtropical Dry Broadleaf Forests,23.526622,"MULTIPOLYGON (((-158.01244 21.42219, -157.9915...",Honolulu; Waipahu; Pearl City; Aiea,HI
4,2,34.923123,-120.434372,United States,Santa Maria,Santa Maria,0.312846,54.450694,0.040129,23.64,...,42.000805,340.967420,HIC,MDR,4.174295e+09,"Mediterranean Forests, Woodlands, and Scrub",14.718191,"MULTIPOLYGON (((-120.46375 34.98933, -120.4411...",Santa Maria,CA
6,4,34.427664,-119.743693,United States,Santa Barbara,Santa Barbara,0.362785,59.576284,0.061348,36.50,...,38.101749,332.032274,HIC,MDR,4.159702e+09,"Mediterranean Forests, Woodlands, and Scrub",15.376907,"MULTIPOLYGON (((-119.82444 34.45783, -119.8131...",Santa Barbara,CA
8,6,36.688991,-121.640831,United States,Salinas,Salinas,0.339631,53.886276,0.076114,24.61,...,41.044956,274.027026,HIC,MDR,4.813837e+09,"Mediterranean Forests, Woodlands, and Scrub",15.274110,"MULTIPOLYGON (((-121.66750 36.74127, -121.6560...",Salinas,CA
9,7,34.217486,-119.209132,United States,Oxnard,Oxnard,0.299903,135.224578,0.036199,28.65,...,97.043526,325.861123,HIC,MDR,1.074582e+10,"Mediterranean Forests, Woodlands, and Scrub",17.053577,"MULTIPOLYGON (((-119.31772 34.29254, -119.2952...",Oxnard; Ventura,CA


Seems updated green_us has more rows than merged dataset. 

It's acutally OK if the number of rows of green_us doesn't match of merged dataset, as long as we have holistic geometry data that can present the all cities areas of merged dataset.

So we need to make sure we got enough geometry data in updated green_us by comparing the unique number of UC_Grouping of both datasets.

In [13]:
n_mer = len(uc_merged['UC_Grouping'].unique())
n_green = len(up_greenus['UC_Grouping'].unique())

if n_mer == n_green:
    print(f"The numbers of unique UC_Grouping are the same, which is {n_mer}.")
else:
    print(f"Numbers are not the same. Merged datasets has {n_mer} unique UC_Grouping, Green_us has {n_green} uniques UC_Grouping")

The numbers of unique UC_Grouping are the same, which is 228.


Since we have covered all cities, we can just keep 1 geometry data point for 1 unique UC_Grouping and also remove less relevant data to make the file size smaller.


In [14]:
def smaller_file(gdf, key_cols = ['UC_Grouping','geometry']):
    df = gdf[key_cols]
    return df.drop_duplicates()

In [15]:
ess_geo = smaller_file(up_greenus)
print(ess_geo.shape)
ess_geo.head()

(228, 2)


,UC_Grouping,geometry
0,0,"MULTIPOLYGON (((-158.01244 21.42219, -157.9915..."
4,2,"MULTIPOLYGON (((-120.46375 34.98933, -120.4411..."
6,4,"MULTIPOLYGON (((-119.82444 34.45783, -119.8131..."
8,6,"MULTIPOLYGON (((-121.66750 36.74127, -121.6560..."
9,7,"MULTIPOLYGON (((-119.31772 34.29254, -119.2952..."


We have cleaned less relevant rows and columns, we are ready to output the geojson file.

In [19]:
# output cleaned geometry df to geojson
_ = gcm.bound_load_file_output_geojson(file_path=_, df=ess_geo, full_state=True, output=True, output_folder='', output_filename='Greenspace_US.geojson')

Be aware of large dataset!


# Choropleth Map for Merged Dataset

Now we will create a choropleth map to visualize features geographically.

In [17]:
# check the geojson file
f = open('Greenspace_US.geojson', 'r')
f.readlines()[:10]

['{\n',
 '"type": "FeatureCollection",\n',
 '"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },\n',
 '"features": [\n',
 '{ "type": "Feature", "properties": { "UC_Grouping": 0 }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ -158.012436513808552, 21.422192591895211 ], [ -157.991578163004789, 21.422192591895211 ], [ -157.986010505889595, 21.413880829006239 ], [ -157.944295274387656, 21.413880829006239 ], [ -157.938731796317313, 21.405569246135077 ], [ -157.928303355789296, 21.405569246135077 ], [ -157.91163026842824, 21.380635576791448 ], [ -157.901202928870106, 21.380635576791448 ], [ -157.890101799975866, 21.3640140291173 ], [ -157.879675193504255, 21.3640140291173 ], [ -157.87412905162148, 21.355703524595818 ], [ -157.853276571229003, 21.355703524595818 ], [ -157.842193859673387, 21.339083053766089 ], [ -157.831768351492457, 21.339083053766089 ], [ -157.826231413465422, 21.330773087290719 ], [ -157.80538112858352, 21.330773087290719 ]

In [18]:
# create the choropleth map

import folium
import json
lat=39.5 
lon=-98.35
geo_col=['UC_Grouping', 'MHLTH_AdjPrev']
key='feature.properties.UC_Grouping'
color='YlGnBu'
opacity=0.7
weight=1
zoom_start=3
legend='Average Mental Health Prevalence (%)'

boundary_file = 'Greenspace_US.geojson'
df = uc_merged

m = folium.Map(location=[lat, lon], zoom_start=zoom_start)

geodata = json.load(open(boundary_file, 'r'))

cp = folium.Choropleth(
    geo_data=geodata,
    data=df,
    columns=geo_col,
    key_on=key,
    fill_color=color,
    fill_opacity=opacity,
    line_weight=weight,
    legend_name=legend
).add_to(m)


display(m)
